In [1]:
# 목적: 
 # 프로세스를 여러개 띄워서 ES에 데이터 반복 입력
 # 성능 확인

# 입력
 # - 프로세스의 개수
 # 각 프로세스의 입력
 # - ES 도큐먼트 번호 시작 초기값(base_index)
 # - ES에 저장할 document 개수 입력받기

# 수행 후 결과 :
 # ES에 데이터 입력
 # 각 프로세스 종료시, 아래의 내용 출력
 # - 1건 데이터 입력 평균시간
 # - 총 데이터 입력 시간, 총 데이터 입력 건수

In [2]:
# 1. 프로세스 개수 입력받기
# 2. ES에 저장할 document 개수 입력받기
# 3. 프로세스 개수만큼 프로세스 띄우기 
# 4. ES 데이터 저장
# 5. multi process 수행 
# 6. 각 프로세스 종료 시, 
#  ① 1건 데이터 입력 평균시간, ② 총 데이터 입력 시간, ③ 총 데이터 입력 건수 출력

In [3]:
import numpy as np

import requests
import json

import multiprocessing as mp
import time

In [4]:
def get_sample_noise(noise_file_name):
    
    # 소음데이터 txt 파일 읽기
    # 파일을 읽어 리스트로 만듦
    lines = []
    with open(noise_file_name, "r") as noise_file:
        lines = noise_file.readlines()
    
    # 랜덤한 위치에서 10000 * 60건 읽기   
    start_indices = []
    start_indices.append(np.random.randint(5400000) + 1) # 1은 파일의 앞쪽, 데이터가 아닌 부분을 제거하기 위한 오프셋
    
    # 시작 위치 인덱스 부터 600000개 데이터 추출
    for start_index in start_indices:
        
        bundle = lines[start_index:start_index + 600000]     
        
        bundle_data = []
        for line in bundle:
            # 소음데이터 사이에 개행문자를 제거하기
            stripped = line.rstrip()
            # index열 제거하기
            bundle_data.append(stripped.split(',')[1])
           
    return bundle_data


In [5]:
# 1. 프로세스 개수 입력받기
process_cnt = int(input("프로세스 개수 입력 : "))
print("입력받은 프로세스 개수 : {}".format(process_cnt))

프로세스 개수 입력 : 30
입력받은 프로세스 개수 : 30


In [6]:
# 4. ES 데이터 저장
def save_es(doc_cnt, q):
    
    # 소음데이터 파일
    noise_txt_file = '/src/data/NoiseRawData_10000/0/r_10min.txt'

    noise_list = []
    
    #print("read file: %s" %noise_txt_file)
    sample_noise_list = get_sample_noise(noise_txt_file)
    noise_list = sample_noise_list
    #print("read complete.")
    
    # 'es-raw' 인덱스에 스페이스(' ')를 넣어 하나의 String으로 저장하기
    string_data = ' '.join(noise_list)
    
    headers = {'Content-Type': 'application/json; charset=utf-8'}

    doc = {
            'type': 'text',
            'index': 'false',
            'text': string_data
    }
    
    data = json.dumps(doc)
    
    # data 파일로 저장
    #file = open('/src/hyebin/esnoise/esnoise_json_data(100).txt', 'w')
    #file.write(data)  
    #file.close()
    
    time_list = []
    for i in doc_cnt:
        
        print("index : {}".format(i))
        
        # 시작시간
        start_time = time.time()
               
        # https://quickstart-es-http.default:9200
        # http://192.168.10.204:9200
        response = requests.post(url = "https://quickstart-es-http.default:9200/es-raw/_doc/",
                                headers = headers,
                                data = data,
                                verify = False)
        # 종료시간
        end_time = time.time()
        
        time_list.append(end_time - start_time)
        
        #print("data length : ", len(data))
        #print("resposne code : ", response.status_code)
        response.raise_for_status()

    time_list_np = np.array(time_list)

    q.put(response)
    
    print("time list : {}".format(time_list_np))

    print("--------------------------------")
    print("데이터 1건 입력 평균 시간 : {}".format(np.mean(time_list_np)))
    

In [7]:
# 2. ES에 저장할 document 개수 입력받기
doc_cnt = int(input("저장할 document 개수 입력 : "))
print("입력받은 document 개수 : {}".format(doc_cnt))
    
# 3. 프로세스 개수만큼 프로세스 띄우기
q_list = [mp.Manager().Queue() for i in range(process_cnt)]

doc_num_list = np.arange(doc_cnt * process_cnt)
print(doc_num_list)
doc_cnt_list = [doc_num_list[i*doc_cnt:(i+1)*doc_cnt] for i in range(process_cnt)]
print(doc_cnt_list)

process_list = [mp.Process(target=save_es, args=(doc_cnt_list[i], q_list[i]), daemon=True) for i in range(process_cnt)]

#print(q_list)
#print(process_list)

저장할 document 개수 입력 : 70
입력받은 document 개수 : 70
[   0    1    2 ... 2097 2098 2099]
[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69]), array([ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139]), array([140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
       153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 

In [8]:
## 5. multi process 수행 

print("Start Processes")
# 시작시간
total_start_time = time.time() 

for process in process_list:
    process.start()

# 프로세스 종료 대기
for process in process_list:
    process.join()

print("***")
print("총 데이터 입력 건수 : {}".format(process_cnt * doc_cnt))
print("총 데이터 입력 시간 : {}".format(time.time() - total_start_time))
print("End Processes")

Start Processes
index : 560
index : 1330


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


index : 770
index : 1540
index : 210


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


index : 350
index : 840
index : 1120
index : 1190


...

time list : [24.47083426 19.96671772 55.56833625 30.50451016 10.81416631 31.69217157
 12.99155164 12.77421236 42.65488625 26.74442124 36.11852884 21.70403385
 19.56273794 21.0225122   6.04075336 21.6664176  36.5081749  49.35057116
  8.24638128  6.79453254  5.80656123 22.03443241 16.94780421 35.45627284
 16.70909071 10.15808606 21.72574043 10.88434625 46.60848308 18.26136351
 18.5230329  13.5261035  20.79116416 10.78774166 48.59494424 40.28182602
 19.06204343 20.97246003 38.9402976   9.52662683 12.76634884 19.34845853
 29.11663008 10.42383599 28.52559543  6.70696402  8.34824395 24.13358188
 19.63663054 48.54225755 23.03185558 15.3693583   9.13118196 17.56640935
 58.00736141  9.30814433  3.95805526  3.09796667  4.03308344  5.54780221
 38.20343733 12.49954963 12.17662358  3.68597198  7.98707891  4.86503315
  3.1885221   1.45863318  3.15704918  2.55036569]
--------------------------------
데이터 1건 입력 평균 시간 : 19.81667003631592
index : 839


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


index : 1679


/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


time list : [20.96311355 30.50201321 38.96257615 26.18356991 30.78816557  6.76050091
 19.73209429 30.92019343 11.39618301 35.84194779  6.53898191  9.14262414
 39.14699411 11.92488384  5.84746528 43.2454536  21.33894587 36.73597836
  6.65061474 72.16369939 10.62678695 40.53320312 32.66954494 30.55422401
 14.32486176 18.77172494 22.69561505 32.21567822 19.76610661 22.36966109
  5.24549437 30.06332564 15.914783    8.25177717 24.05036497 15.16259313
 31.95625019 33.77888274 20.66017866 16.62567949 19.87500334 22.20581985
 34.8513968   9.60487652 20.7919302  22.51487374  7.37266016 20.82384872
 14.60091472 11.02134109  6.6534791  28.80216908 18.47337437 20.30059934
  5.91834354 12.60747409 19.84529901 66.1268518  17.87721848  9.38554358
 13.51564932  7.46142578  4.43166089  4.70325065  5.56956959  1.61228442
  3.72611475  2.95040059  2.21592093  1.86600542]
--------------------------------
데이터 1건 입력 평균 시간 : 19.838971928187778
time list : [17.03564334 11.86039972 19.06749964 30.36431289 10.4